# Autotutorial de preprocesado con _scikit-learn_

En este cuaderno vamos a aprender a ver varios métodos de preprocesado que proporciona scikit-learn

Puedes encontrar más información en la ayuda en-linea de _scikit-learn_

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [1]:
seed = 1234 #<- random generator seed (comment to get randomness)

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/') #_ this is a package by myself with utils
import MyUtils as my           #

Comenzamos cargando datos y agrupándolos en 2 dataframes,  *X_full* e *Y_full*, para poder trabajar sobre ellos.

In [2]:
#- Load data from CSV and put all in a single dataframe 'FullSet'

FullSet_0 = pd.read_csv('../../Datasets/1000ceros.csv', header=None)
FullSet_1 = pd.read_csv('../../Datasets/1000unos.csv',  header=None)
FullSet = my.join_features_labels(FullSet_0,FullSet_1)

#- Convert the 'FullSet' of pixels into the set 'X_full' of features and get the set 'Y_full' of labels

theta = 0.5
X_full = my.mnist_features( FullSet.drop('label', axis=1), theta=theta )
Y_full = FullSet[['label']]

### Separación estratificada con selección aleatoria de elementos

sklearn.model_selection.**StratifiedShuffleSplit**

In [3]:
from sklearn.model_selection import StratifiedShuffleSplit

In [4]:
#- Split X_full into TRAIN and TEST using scikit-learn

test_size = 0.2 #- percentage of instances held for testing
splitter = StratifiedShuffleSplit(n_splits=1,test_size=test_size, random_state=seed)
#-> splitter is an object that will create ONE split, i.e. 2 subsets
split_ix = splitter.split(X_full,Y_full)
#-> method split needs the set of instances and the associated set of labels
#-> it returns the indexes of the instances that go to the first subset and to the second subset
for train_ix, test_ix in split_ix:
    X_train = X_full.loc[train_ix].reset_index(drop=True) # remember that out inputs are dataframes
    Y_train = Y_full.loc[train_ix].reset_index(drop=True) #  not numpy arrays !!
    X_test  = X_full.loc[test_ix].reset_index(drop=True)  # -> that's why we have to write
    Y_test  = Y_full.loc[test_ix].reset_index(drop=True)  #  so much !!

print('X_train shape is ',X_train.shape)
print('Y_train shape is ',Y_train.shape)
print('X_test shape is ',X_test.shape)
print('Y_test shape is ',Y_test.shape)

X_train shape is  (1600, 10)
Y_train shape is  (1600, 1)
X_test shape is  (400, 10)
Y_test shape is  (400, 1)


In [5]:
#- Split MANY TIMES X_full into TRAIN and TEST using scikit-learn

test_size = 0.2 #- percentage of instances held for testing
n_splits  = 2   #- NEW variable !! = number of times we make splitting
splitter = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)
#-> splitter is an object that will create 2 subsets, 'n_splits' times.
split_ix = splitter.split(X_full,Y_full)
#-> method split needs the set of instances and the associated set of labels
#-> it returns the indexes of the instances that go to the first subset and to the second subset

count = 0 
for train_ix, test_ix in split_ix:
    print('count ',count); count+=1
    
    X_train = X_full.loc[train_ix].reset_index(drop=True)
    Y_train = Y_full.loc[train_ix].reset_index(drop=True)
    X_test  = X_full.loc[test_ix].reset_index(drop=True)
    Y_test  = Y_full.loc[test_ix].reset_index(drop=True)
    
    print('X_train shape is ',X_train.shape)
    print('Y_train shape is ',Y_train.shape)
    print('X_test shape is ',X_test.shape)
    print('Y_test shape is ',Y_test.shape)

count  0
X_train shape is  (1600, 10)
Y_train shape is  (1600, 1)
X_test shape is  (400, 10)
Y_test shape is  (400, 1)
count  1
X_train shape is  (1600, 10)
Y_train shape is  (1600, 1)
X_test shape is  (400, 10)
Y_test shape is  (400, 1)


En el resultado de la celda anterior podemos ver como la tarea de separar en dos subconjuntos (tanto los datos como las etiquetas) se ha realiado tantas veces como marca la variable **n_splits**.

Por otra parte, como con cualquier objeto de Python, podemos ver los atributos de **spliter** invocando `.__dict__`, e incluso acceder a ellos, aunque normalmente también hay métodos (_getters_ y _setters_) para eso.

In [6]:
print('Attributes of splitter: ',splitter.__dict__,'\n')
print(splitter.n_splits, '= how many times we are making a split, by reading the attribute \n')
print(splitter.get_n_splits(),'= how many times we are making a split, by using the "get" method \n')

Attributes of splitter:  {'n_splits': 2, 'test_size': 0.2, 'train_size': None, 'random_state': 1234, '_default_test_size': 0.1} 

2 = how many times we are making a split, by reading the attribute 

2 = how many times we are making a split, by using the "get" method 



**Comentarios**

+ Si sólo queremos hacer 1 división, por ej. TRAIN + TEST, es más rápido y sencillo utilizar **train_test_split**.<br>
La división es estratificada.
+ Si queremos hacer una división tipo *K-fold*, entences tenemos que utilizar **StratifiedKFold**, donde el parámetro *n_splits* es el número de divisiones que realizamos sobre el conjunto original. 
+ Consulta en [la ayuda en linea](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) para ver otros métodos como **Leave-One-Out**, etc.

In [7]:
# train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_full,  Y_full, test_size=test_size, random_state=seed)

# StratifiedKFold 
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits)
skf_ix = skf.split(X_full,Y_full)
for train_ix, test_ix in split_ix:    
    X_train = X_full.loc[train_ix].reset_index(drop=True)
    Y_train = Y_full.loc[train_ix].reset_index(drop=True)
    X_test  = X_full.loc[test_ix].reset_index(drop=True)
    Y_test  = Y_full.loc[test_ix].reset_index(drop=True)

### Transformación de características
*Scikit-learn* proporciona una gran cantidad de métodos para transformar los conjuntos de datos.<br>
Puedes consultar todos en la [ayuda en línea de *preprocesado*](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

Los puntos más importantes son:
+ Aquellas transformaciones en las que es necesario calcular algún parámetro para poder realizarlas, se deben hacer sólo sobre el conjunto de entrenamiento. Después esos parámetros se aplican sobre los demas conjuntos de datos.
    > Por ej. Si queremos escalar los datos al intervalo $[0,1]$, tenemos que averiguar cuál es el valor máximo y mínimo del conjunto de entrenamiento, NO del conjunto que contenga todos los datos
    <br> (el conjunto que contiene todos los datos NO existe, sería aquel que contenga todo lo que la máquina va a recibir en el futuro)
+ Hay 3 métodos esenciales: *fit*, *transform* y *fit_transform*
    - **fit** calcula los parámetros de la transformación a partir de los datos
    - **transform** realiza la transformación con los parámetros calculados por fit
    - **fit_transform** primero hace *fit* y luego *transform*

In [8]:
#- Select the features and scale to [0,1] with respect to the TRAIN set

from sklearn.preprocessing import MinMaxScaler

feat_selec= ['width' , 'height']

# Doing "fit" and "transform"
scaler = MinMaxScaler()
scaler.fit(X_train[feat_selec])  #<- fit = get the parameters of the scaling
X_train_unit = scaler.transform(X_train[feat_selec]) #<- transform X_train 
X_test_unit  = scaler.transform(X_test[feat_selec])  #<- transform X_test

# Doing "fit_transform" and "transform"
scaler2 = MinMaxScaler()
X_train_unit2 = scaler2.fit_transform(X_train[feat_selec]) #<-fit & transform X_train in 1 step
X_test_unit2  = scaler2.fit_transform(X_test[feat_selec])  #<- transform X_test

# checking that we get the same outcomes -
if np.sum(X_train_unit - X_train_unit2) == 0 :
    print('X_train_unit = X_train_unit2')
if np.sum(X_test_unit - X_test_unit2) == 0 :
    print('X_test_unit = X_test_unit2')

X_train_unit = X_train_unit2
X_test_unit = X_test_unit2


**Cometarios**
+ Los objetos de sklearn.preprocessing devuelven numpy arrays<br>
  $\rightarrow$ Si queremos seguir trabajando con dataframes de pandas habrá que construirlo a partir de la salida y de las características elegidas
+ Recuerda que `.__dict__` devuelve los atributos del objeto 

In [9]:
# from numpy array to pandas dataframe 
X_train_unit_df = pd.DataFrame(X_train_unit, columns = feat_selec)
X_train_unit_df.head()

,width,height
0,0.789474,0.833333
1,0.526316,0.944444
2,0.157895,0.888889
3,0.684211,0.888889
4,0.526316,1.000000


In [10]:
scaler.__dict__

{'feature_range': (0, 1),
 'copy': True,
 'n_features_in_': 2,
 'n_samples_seen_': 1600,
 'scale_': array([0.05263158, 0.05555556]),
 'min_': array([ 0.        , -0.05555556]),
 'data_min_': array([0., 1.]),
 'data_max_': array([19., 19.]),
 'data_range_': array([19., 18.])}

## Ejercicio
1. Crear un conjunto de datos utilizando las características **W_max1** y **area** 
2. Realizar un 5-fold de tal manera que:
    - En cada iteración se escalen las características respecto al conjunto de entrenamiento al intervalo $[0,1]$
    - Se aprenda un clasificador lineal por regresión logística
    - Se evalue clasificador con el conjunto de test

### Solución 

**Comentarios**
+ Normalmente los resultados de un fold a otro varían "ligeramente", **¿con cuál me quedo?**
+ Lo mejor es quedarse con aquel modelo que esté más próximo al promedio, ni con el "mejor" ni con el "peor".<br>
  $\rightarrow$ es conveniente guardar el modelo de cada _fold_ en una lista, para después recuperar el elegido.
+ <u>Importante.</u> Al hacer validación cruzada no necesitamos separar el conjunto de datos inicial en *TRAIN-VALIDATION-TEST* porque el mismo procedimiento va obtiendo *TRAIN* y *VALIDATION* en cada _fold_.
